# D-DUST Data Grid Processing Notebook

- - -

Check the following link for Data Management Plan and the Variable list table: <br>
**1. [Data Management Plan (DMP)](https://docs.google.com/document/d/1n3PVat7PBTG76JnINOkL2pvBZuKQlakZkTgqNj39oAQ/edit#)**<br>
**2. [Variables list table](https://docs.google.com/spreadsheets/d/1-5pwMSc1QlFyC8iIaA-l1fWhWtpqVio2/edit#gid=91313358)**

This notebook describes the physical variables selected for the project and how they are preprocessed.
These variables are divided into <u>4 categories</u>, as shown in the Variable list table:
1. **Map Layer**: static layer used to describe Lombardy region morphology and its features (such as elevation, infrastructures, land use and cover etc.)
2. **Model**: data retrieved from a model that uses satellite and in-situ observations of meteorological and air quality data as input (such as ERA5, CAMS).
3. **Satellite**: data obtained directly from satellite observations (such as Sentinel-5P).
4. **Ground Sensor**: data retrieved from ground monitoring stations measuring air quality and meteorological variables.

First the other notebooks must be used in order to retrieve all the data in the same time range.
- GEE_API Notebook: satellite images + ERA5 data
- ADS_API Copernicus Notebook: ADS data from CAMS analysis model
- ARPA_API Ground Sensors Notebook: data from ARPA ground sensor
- AQ_ESA Stations Notebook: data from ESA Air Quality station


## Import libraries

In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
import scipy.interpolate
from scipy.interpolate import griddata
import rasterstats as rstat
import rioxarray
import shapely.speedups
shapely.speedups.enable()
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from shapely.geometry import shape
from shapely.geometry import  MultiLineString
from rasterio.enums import Resampling
import ipywidgets as widgets

import warnings
warnings.filterwarnings('ignore')

C:\Users\Administrator\anaconda3\envs\ddust\lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
from functions import my_methods

In [3]:
cwd = os.getcwd()

## Import grids

Three grids with different spatial resolution are used in this project:
1. **cams** grid: 0.1° x 0.1° resolution - Grid with CAMS Model spatial resolution.
2. **s5p** grid: 0.066° x 0.066° resolution - Grid with the Sentinel-5P approximate spatial resolution.
3. **arpa** grid: 0.01° x 0.01° resolution- Grid generated with at most one ARPA monitoring station for each pixel.

These grids are defined as bounding box of the Lombardy region layer applying a buffer of 20 km.

In [4]:
# With this widget is possible to select from the dropdown list
name = widgets.Dropdown(
    options=['cams', 's5p', 'arpa'],
    description='name:',
    disabled=False)
name

Dropdown(description='name:', options=('cams', 's5p', 'arpa'), value='cams')

In [5]:
#to interpolate CAMS .tif file with a lower resolution depending on the grid cell size (useful for zonal statistics)
if name.value == 'cams':
    upscale_factor = 1  
if name.value == 's5p':
    upscale_factor = 10
if name.value == 'arpa':
    upscale_factor = 10  

grid_path = my_methods.select_grid(name.value)

In [6]:
grid = gpd.read_file(cwd + grid_path)

In [7]:
m_to_km = 10**(-3)
m2_to_km2 = 10**(-6)

---

# Importing Map Layers

### [DUSAF - Land use - Geoportale Lombardia](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7B18EE7CDC-E51B-4DFB-99F8-3CF416FC3C70%7D) <br>

Consists in a multi-temporal geographic database that classifies land based on major land cover and land use types. Reference system EPSG:4326.<br>
Land use:
- 2 = Aree agricole.
- 3 =Territori boscati e ambienti seminaturali.
- 4 = Aree umide.
- 5 = Corpi idrici.
- 11 = Zone urbanizzate.
- 12 = Insediamenti produttivo, grandi impianti e reti di comunicazione.
- 13 = Aree estrattive, discariche, cantieri, terreni artefatti e abbandonati.
- 14 = Aree verdi non agricole.

In [8]:
dusaf_path = cwd + '/land_use_cover/DUSAF6_dissolve_rast_4326.tiff'

In [9]:
dusaf = rio.open(dusaf_path)
dusaf_array = dusaf.read(1).astype('float64') 
dusaf_array[dusaf_array<1.0]=np.nan
affine = dusaf.transform

In [10]:
# Class majority calculation for each cell
stats = rstat.zonal_stats(grid, dusaf_array, affine=affine, nodata=np.nan, stats=['majority'], categorical=True)
majority_list = [{k: v for k, v in d.items() if k == 'majority'} for d in stats]
grid = grid.join(pd.DataFrame(majority_list), how='left')
grid = grid.rename(columns={"majority": "dusaf"})

In [11]:
# Class counts in each tile
stats = rstat.zonal_stats(grid, dusaf_array, affine=affine, nodata=np.nan, stats=['count'], categorical=True)
p = pd.DataFrame.from_dict(stats, orient='columns')
p = p*m2_to_km2

grid['dsf2'] = p[2.0]
grid['dsf3'] = p[3.0]
grid['dsf4'] = p[4.0]
grid['dsf5'] = p[5.0]
grid['dsf11'] = p[11.0]
grid['dsf12'] = p[12.0]
grid['dsf13'] = p[13.0]
grid['dsf14'] = p[14.0]
grid['dsfSum'] = p['count']

 - - -

### [SIARL - Agricultural use - Geoportale Lombardia](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7B83483117-8742-4A1F-A16E-3A48AEE2EBE2%7D) <br>
This layer contains the agricoltural use for each cadastral parcel provided by SIARL 2019 Catalog for the Lombardy region. Reference system EPSG:4326. <br>
Agricoltural use:
- 2 Altri cereali
- 9 Mais
- 12 Riso

In [12]:
siarl_path = cwd + '/land_use_cover/siarl.tif'
siarl = rio.open(siarl_path)

In [13]:
siarl_array = siarl.read(1).astype('float64') 
siarl_array[siarl_array<1.0]=np.nan
affine = siarl.transform

In [14]:
# Class majority calculation for each cell
stats_siarl = rstat.zonal_stats(grid, siarl_array, affine=affine, nodata=np.nan, stats=['majority'], categorical=True)
majority_list = [{k: v for k, v in d.items() if k == 'majority'} for d in stats_siarl]
grid = grid.join(pd.DataFrame(majority_list), how='left')
grid = grid.rename(columns={"majority": "siarl"})

In [15]:
# Class counts in each tile
stats_siarl = rstat.zonal_stats(grid, siarl_array, affine=affine, nodata=np.nan, stats=['count'], categorical=True)
p = pd.DataFrame.from_dict(stats_siarl, orient='columns')
p = p*m2_to_km2

grid['siarl2'] = p[2.0]
grid['siarl9'] = p[9.0]
grid['siarl12'] = p[12.0]

grid['siarlSum'] = p['count']

- - -

### [Digital Terrain Model - Geoportale Lombardia](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7BFC06681A-2403-481F-B6FE-5F952DD48BAF%7D)<br>
Digital Terrain Model of Lombardy region with 20 m resolution. Reference system EPSG:4326.
1. **Elevation** = DTM with 20 m resolution
2. **Aspect** = calculated previously from the elevation layer
3. **Slope** = calculated previously from the elevation layer

In [16]:
dtm_path = cwd + '/terrain/dtm20.tif'
aspect_path = cwd + '/terrain/aspect.tif'
slope_path = cwd + '/terrain/slope.tif'
dtm = rio.open(dtm_path)
aspect = rio.open(aspect_path)
slope = rio.open(slope_path)

In [17]:
#Calculates mean value in each cell for the elevation
dtm_array = dtm.read(1)
dtm_array[dtm_array<0]=np.nan
affine = dtm.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, dtm_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "h_mean"})

In [18]:
#Calculates mean value in each cell for aspect
aspect_array = aspect.read(1)
aspect_array[aspect_array<0]=np.nan
affine = aspect.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, aspect_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "aspect_mean"})

In [19]:
#Calculates mean value in each cell for slope
slope_array = slope.read(1)
slope_array[slope_array<0]=np.nan
affine = slope.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, slope_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "slope_mean"})

---

### Soil and Vegetation

In this part information concerning soil and vegetation is considered, such as:
- Soil type: classification fron OpenLandMap soil classification
- Soil text: classification obtained from (Carta Pedologica 1:250.000) from Geoportale Regione Lombardia
- Soil moist: soil moisture in soil layer 1 (0 - 7 cm) of the ECMWF Integrated Forecasting System
- NDVI: NDVI obtained from Terra Vegetation Indices 16-Day Global 250m dataset

USDA Soil Classification (missing value 3, 10, 12 in this Lombardy region): 
- 1 Clay
- 2 Silty Clay
- 3 Sandy Clay
- 4 Clay Loeam
- 5 Silty Clay Loam
- 6 Sandy Clay Loam
- 7 Loam
- 8 Silt Loam
- 9 Sandy Loam
- 10 Silt
- 11 Loamy Sand
- 12 Sand

In [20]:
soil_type_path = cwd + '/temp/soil_type.tif'
soil_moist_path = cwd + '/temp/soil_hum.tif'
ndvi_path = cwd + '/temp/ndvi.tif'
soil_text_path = cwd + '/terrain/carta_pedologica_4326_dissolved_250K.tif'

soil_type = rio.open(soil_type_path)
soil_moist = rio.open(soil_moist_path)
ndvi = rio.open(ndvi_path)
soil_text = rio.open(soil_text_path)

In [21]:
soil_type_array = soil_type.read(1).astype('float64') 
soil_type_array[soil_type_array<1.0]=np.nan
affine = soil_type.transform

In [22]:
stats_soil_type = rstat.zonal_stats(grid, soil_type_array, affine=affine, nodata=np.nan, stats=['majority'], categorical=True)
majority_list = [{k: v for k, v in d.items() if k == 'majority'} for d in stats_soil_type]
grid = grid.join(pd.DataFrame(majority_list), how='left')
grid = grid.rename(columns={"majority": "soil"})

Soil type from Open Land Map:

In [23]:
# Class counts in each tile
stats_soil = rstat.zonal_stats(grid, soil_type_array, affine=affine, nodata=np.nan, stats=['count'], categorical=True)
p = pd.DataFrame.from_dict(stats_soil, orient='columns')
grid['soil1'] = p[1.0]
grid['soil2'] = p[2.0]
#grid['soil3'] = p[3.0] #not present in lombardy region .tif
grid['soil4'] = p[4.0]
grid['soil5'] = p[5.0]
grid['soil6'] = p[6.0]
grid['soil7'] = p[7.0]
grid['soil8'] = p[8.0]
grid['soil9'] = p[9.0]
#grid['soil10'] = p[10.0] #not present in lombardy region .tif
grid['soil11'] = p[11.0]
#grid['soil12'] = p[12.0] #not present in lombardy region .tif

grid['soilSum'] = p['count']

Soil texture type from Carta Pedologica Regione Lombardia:
- 1 Argillosa fine
- 2 Argillosa molto fine
- 3 Franca fine
- 4 Franca grossolana
- 5 Limosa fine
- 6 Limosa grossolana
- 7 Sabbiosa
- 8 Scheletrico-Argillosa
- 9 Scheletrico-Franca
- 10 Scheletrico-Sabbiosa

In [24]:
soil_text_array = soil_text.read(1).astype('float64') 
soil_text_array[soil_text_array<1.0]=np.nan
affine = soil_text.transform

In [25]:
stats_soil_text = rstat.zonal_stats(grid, soil_text_array, affine=affine, nodata=np.nan, stats=['majority'], categorical=True)
majority_list = [{k: v for k, v in d.items() if k == 'majority'} for d in stats_soil_text]
grid = grid.join(pd.DataFrame(majority_list), how='left')
grid = grid.rename(columns={"majority": "soil_text"})

In [26]:
# Class counts in each tile
stats_soil_text = rstat.zonal_stats(grid, soil_text_array, affine=affine, nodata=np.nan, stats=['count'], categorical=True)
p = pd.DataFrame.from_dict(stats_soil_text, orient='columns')
grid['soil_text1'] = p[1.0]
grid['soil_text2'] = p[2.0]
grid['soil_text3'] = p[3.0]
grid['soil_text4'] = p[4.0]
grid['soil_text5'] = p[5.0]
grid['soil_text6'] = p[6.0]
grid['soil_text7'] = p[7.0]
grid['soil_text8'] = p[8.0]
grid['soil_text9'] = p[9.0]
grid['soil_text10'] = p[10.0]

grid['soilTextSum'] = p['count']

Soil Moisture from ERA5 model:

In [27]:
soil_moist_array = soil_moist.read(1)
affine = soil_moist.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, soil_moist_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "soil_moist"})

NDVI from MODIS 16-days global:

In [28]:
ndvi_array = ndvi.read(1)
affine = ndvi.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, ndvi_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "ndvi"})

---

### Import climate:

In [29]:
# Set paths
temp_2m_path = cwd + '/temp/temp_2m.tif'
prec_path = cwd + '/temp/prec.tif'
press_path = cwd + '/temp/press.tif'
n_wind_path = cwd + '/temp/n_wind.tif'
e_wind_path = cwd + '/temp/e_wind.tif'

temp_2m = rio.open(temp_2m_path)
prec = rio.open(prec_path)
press = rio.open(press_path)
n_wind = rio.open(n_wind_path)
e_wind = rio.open(e_wind_path)

In [30]:
temp_2m_array = temp_2m.read(1)
affine = temp_2m.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, temp_2m_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "temp_2m"})

In [31]:
prec_array = prec.read(1)
prec_array[prec_array<0]=np.nan
affine = prec.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, prec_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "prec"})

In [32]:
press_array = press.read(1)
press_array[press_array<0]=np.nan
affine = press.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, press_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "press"})

In [33]:
n_wind_array = n_wind.read(1)
affine = n_wind.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, n_wind_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "n_wind"})

In [34]:
e_wind_array = e_wind.read(1)
affine = e_wind.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, e_wind_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "e_wind"})

### Import pollutants

In [35]:
#From S5P
no2_path = cwd + '/temp/no2.tif'
so2_path = cwd + '/temp/so2.tif'
aod55_path = cwd + '/temp/aod_055.tif'
aod47_path = cwd + '/temp/aod_047.tif'
uvai_path = cwd + '/temp/uvai.tif'
co_path = cwd + '/temp/co.tif'
form_path = cwd + '/temp/form.tif'
ozone_path = cwd + '/temp/o3.tif'

no2 = rio.open(no2_path)
so2 = rio.open(so2_path)
aod55 = rio.open(aod55_path)
aod47 = rio.open(aod47_path)
uvai = rio.open(uvai_path)
co = rio.open(co_path)
form = rio.open(form_path)
ozone = rio.open(ozone_path)

#From CAMS
amm_cams_path = cwd + '/temp/amm_cams.nc'
no_cams_path = cwd + '/temp/no_cams.nc'
co_cams_path = cwd + '/temp/co_cams.nc'
no2_cams_path = cwd + '/temp/no2_cams.nc'
dust_cams_path = cwd + '/temp/dust_cams.nc'
pm10_cams_path = cwd + '/temp/pm10_cams.nc'
pm25_cams_path = cwd + '/temp/pm25_cams.nc'
nmvocs_cams_path = cwd + '/temp/nmvocs_cams.nc'
so2_cams_path = cwd + '/temp/so2_cams.nc'
ozone_cams_path = cwd + '/temp/o3_cams.nc'

amm_cams = rioxarray.open_rasterio(amm_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
no_cams = rioxarray.open_rasterio(no_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
co_cams = rioxarray.open_rasterio(co_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
no2_cams = rioxarray.open_rasterio(no2_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
dust_cams = rioxarray.open_rasterio(dust_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
pm10_cams = rioxarray.open_rasterio(pm10_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
pm25_cams = rioxarray.open_rasterio(pm25_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
nmvocs_cams = rioxarray.open_rasterio(nmvocs_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
so2_cams = rioxarray.open_rasterio(so2_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)
ozone_cams = rioxarray.open_rasterio(ozone_cams_path,masked=True).rio.write_crs("epsg:4326", inplace=True)

#Convert ammonia to .tif
amm_cams.rio.to_raster(cwd + "/temp/amm_cams.tif")
amm_cams_path = cwd + '/temp/amm_cams.tif'
amm_cams = rio.open(amm_cams_path)
#Convert NO to .tif
no_cams.rio.to_raster(cwd + "/temp/no_cams.tif")
no_cams_path = cwd + '/temp/no_cams.tif'
no_cams = rio.open(no_cams_path)
#Convert CO to .tif
co_cams.rio.to_raster(cwd + "/temp/co_cams.tif")
co_cams_path = cwd + '/temp/co_cams.tif'
co_cams = rio.open(co_cams_path)
#Convert dust to .tif
dust_cams.rio.to_raster(cwd + "/temp/dust_cams.tif")
dust_cams_path = cwd + '/temp/dust_cams.tif'
dust_cams = rio.open(dust_cams_path)
#Convert pm10 to .tif
pm10_cams.rio.to_raster(cwd + "/temp/pm10_cams.tif")
pm10_cams_path = cwd + '/temp/pm10_cams.tif'
pm10_cams = rio.open(pm10_cams_path)
#Convert pm25 to .tif
pm25_cams.rio.to_raster(cwd + "/temp/pm25_cams.tif")
pm25_cams_path = cwd + '/temp/pm25_cams.tif'
pm25_cams = rio.open(pm25_cams_path)
#Convert NO2 to .tif
no2_cams.rio.to_raster(cwd + "/temp/no2_cams.tif")
no2_cams_path = cwd + '/temp/no2_cams.tif'
no2_cams = rio.open(no2_cams_path)
#Convert NMVOCs to .tif
nmvocs_cams.rio.to_raster(cwd + "/temp/nmvocs_cams.tif")
nmvocs_cams_path = cwd + '/temp/nmvocs_cams.tif'
nmvocs_cams = rio.open(nmvocs_cams_path)
#Convert SO2 to .tif
so2_cams.rio.to_raster(cwd + "/temp/so2_cams.tif")
so2_cams_path = cwd + '/temp/so2_cams.tif'
so2_cams = rio.open(so2_cams_path)
#Convert Ozone to .tif
ozone_cams.rio.to_raster(cwd + "/temp/o3_cams.tif")
ozone_cams_path = cwd + '/temp/o3_cams.tif'
ozone_cams = rio.open(ozone_cams_path)

### Sentinel-5P Data

In [36]:
no2_array = no2.read(1)
affine = no2.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, no2_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "no2_s5p"})

In [37]:
so2_array = so2.read(1)
affine = so2.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, so2_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "so2_s5p"})

In [38]:
aod55_array = aod55.read(1)
affine = aod55.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, aod55_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "aod_055"})

In [39]:
aod47_array = aod47.read(1)
affine = aod47.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, aod47_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "aod_047"})

In [40]:
uvai_array = uvai.read(1)
affine = uvai.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, uvai_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "uvai"})

In [41]:
co_array = co.read(1)
affine = co.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, co_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "co_s5p"})

In [42]:
form_array = form.read(1)
affine = form.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, form_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "form_s5p"})

In [43]:
ozone_array = ozone.read(1)
affine = ozone.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, ozone_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "ozone_s5p"})

### CAMS Data

Resample CAMS data (original resolution 0.1°): zonal stats has issues with original resolution

In [44]:
for files in os.listdir(cwd+'/temp'):
    if files[-9:] == '_cams.tif':
        file_name = files[:(len(files)-9)]
        xds = rioxarray.open_rasterio(cwd + "/temp/"+files,masked=True)
        new_width = xds.rio.width * upscale_factor
        new_height = xds.rio.height * upscale_factor
        xds_upsampled = xds.rio.reproject(xds.rio.crs,shape=(new_height, new_width),resampling=Resampling.bilinear)
        xds_upsampled.rio.to_raster(cwd +'/temp/'+file_name+'_cams_upsampled.tif')

In [45]:
amm_cams_path = cwd + '/temp/amm_cams_upsampled.tif'
amm_cams = rio.open(amm_cams_path)

no_cams_path = cwd + '/temp/no_cams_upsampled.tif'
no_cams = rio.open(no_cams_path)

co_cams_path = cwd + '/temp/co_cams_upsampled.tif'
co_cams = rio.open(co_cams_path)

dust_cams_path = cwd + '/temp/dust_cams_upsampled.tif'
dust_cams = rio.open(dust_cams_path)

pm10_cams_path = cwd + '/temp/pm10_cams_upsampled.tif'
pm10_cams = rio.open(pm10_cams_path)

pm25_cams_path = cwd + '/temp/pm25_cams_upsampled.tif'
pm25_cams = rio.open(pm25_cams_path)

no2_cams_path = cwd + '/temp/no2_cams_upsampled.tif'
no2_cams = rio.open(no2_cams_path)

nmvocs_cams_path = cwd + '/temp/nmvocs_cams_upsampled.tif'
nmvocs_cams = rio.open(nmvocs_cams_path)

so2_cams_path = cwd + '/temp/so2_cams_upsampled.tif'
so2_cams = rio.open(so2_cams_path)

ozone_cams_path = cwd + '/temp/o3_cams_upsampled.tif'
ozone_cams = rio.open(ozone_cams_path)

In [46]:
amm_cams_array = amm_cams.read(1)
affine = amm_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, amm_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "amm_cams"})

In [47]:
no_cams_array = no_cams.read(1)
affine = no_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, no_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "no_cams"})

In [48]:
co_cams_array = co_cams.read(1)
affine = co_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, co_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "co_cams"})

In [49]:
dust_cams_array = dust_cams.read(1)
affine = dust_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, dust_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "dust_cams"})

In [50]:
pm10_cams_array = pm10_cams.read(1)
affine = pm10_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, pm10_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "pm10_cams"})

In [51]:
pm25_cams_array = pm25_cams.read(1)
affine = pm25_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, pm25_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "pm25_cams"})

In [52]:
no2_cams_array = no2_cams.read(1)
affine = no2_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, no2_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "no2_cams"})

In [53]:
nmvocs_cams_array = nmvocs_cams.read(1)
affine = nmvocs_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, nmvocs_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "nmvocs_cams"})

In [54]:
so2_cams_array = so2_cams.read(1)
affine = so2_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, so2_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "so2_cams"})

In [55]:
ozone_cams_array = ozone_cams.read(1)
affine = ozone_cams.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, ozone_cams_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "ozone_cams"})

### [Gridded Population of the World - GPW](https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-rev11)<br>
To provide estimates of population density for the year 2020, based on counts consistent with national censuses and population registers, as raster data to facilitate data integration.
Input reference system EPSG: 4326

In [56]:
pop_path = cwd + '/population/population.tif'
pop = rio.open(pop_path)

In [57]:
pop_array = pop.read(1)
pop_array[pop_array<0]=np.nan
affine = pop.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, pop_array, affine=affine, nodata=np.nan, stats=['sum'])), how='left')
grid = grid.rename(columns={"sum": "pop"})

 - - -

### [Road Infrastructures - Geoportale Lombardia (DBTR 2019)](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7B17D4656F-2E9D-4951-9DC1-4AD32C0959B1%7D): 

**Point layers** considered:
1. Intersection between primary roads including highways
2. Intersection between primary and secondary roads
3. Intersection between secondary roads

Input reference system EPSG: 4326

In [58]:
int_prim_path = cwd + '/road_infrastructures/inters_highway_prim_road.gpkg'
int_prim_sec_path = cwd + '/road_infrastructures/inters_prim_sec_road.gpkg'
int_sec_path = cwd + '/road_infrastructures/inters_sec_road.gpkg'

grid = grid.to_crs(32632)

In [59]:
int_prim = gpd.read_file(int_prim_path).to_crs(32632)
int_prim_sec = gpd.read_file(int_prim_sec_path).to_crs(32632)
int_sec = gpd.read_file(int_sec_path).to_crs(32632)

df_dict = {'int_prim':int_prim,
          'int_prim_sec':int_prim_sec, 'int_sec': int_sec}


for key in df_dict:
    poor_points = df_dict[key][['OBJECTID','geometry']]
    sjoined = gpd.sjoin(poor_points, grid)
    df_count = pd.DataFrame(sjoined.groupby('index_right').size()) 
    grid_join = grid.join(df_count)
    grid[key] = grid_join[0]

**Line layers** considered:
1. Highways
2. Primary roads
3. Secondary roads

Input reference system EPSG: 4326

In [60]:
highway_path = cwd + '/road_infrastructures/highway.gpkg'
prim_road_path = cwd + '/road_infrastructures/prim_road.gpkg'
sec_road_path = cwd + '/road_infrastructures/sec_road.gpkg'

It is required to convert to a cartographic reference system EPSG:32632 to calculate distances.

In [61]:
highway = gpd.read_file(highway_path).to_crs(32632)
prim_road = gpd.read_file(prim_road_path).to_crs(32632)
sec_road = gpd.read_file(sec_road_path).to_crs(32632)

In [62]:
df_dict = {'highway':highway, 'prim_road':prim_road, 'sec_road':sec_road}

for key in df_dict:
    grid[key] = np.nan
    poor_lines = df_dict[key][['geodb_oid','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_lines, mask) 
        l = clip.geometry.length.sum()
        grid[key].iloc[index] = l*m_to_km
    print(key)

highway
prim_road
sec_road


 - - -

### Farms
Vector file obtained from DUSAF 2018 (features with cod. 12112 = "Insediamenti produttivi agricoli
Sono compresi in questa classe gli edifici utilizzati per le attività produttive del settore primario, come capannoni, rimesse per macchine agricole, fienili, stalle, silos, ecc, unitamente agli spazi accessori. Quando tali edifici sono presenti insieme a quelli residenziali configurando un aggregato rurale, se le due tipologie non risultano separabili in modo evidente si classifica tutto il nucleo come cascina (11231)").

In [63]:
farms_path = cwd + '/farms/farms_dissolve.gpkg'
farms = gpd.read_file(farms_path)

In [64]:
df_dict2 = {'farms':farms}

In [65]:
for key in df_dict2:
    grid[key] = np.nan
    poor_poly = df_dict2[key][['COD_TOT','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_poly, mask) 
        a = clip.geometry.area.sum()
        grid[key].iloc[index] = a*m2_to_km2
    print(key)

farms


In [66]:
grid = grid.to_crs(4326)

### Breeding farm type

In [67]:
pigs_path = cwd + '/farms/pigs.gpkg'
poultry_path = cwd + '/farms/poultry.gpkg'
sheep_path = cwd + '/farms/sheep.gpkg'

In [68]:
farm_pigs = gpd.read_file(pigs_path)
farm_poultry = gpd.read_file(poultry_path)
farm_sheep = gpd.read_file(sheep_path)

df_dict = {'farm_pigs':farm_pigs,
          'farm_poultry':farm_poultry, 'farm_sheep':farm_sheep}


for key in df_dict:
    poor_points = df_dict[key][['OBJECTID','geometry']]
    sjoined = gpd.sjoin(poor_points, grid)
    df_count = pd.DataFrame(sjoined.groupby('index_right').size()) 
    grid_join = grid.join(df_count)
    grid[key] = grid_join[0]
    print(key)

farm_pigs
farm_poultry
farm_sheep


---

# Import air quality and meteo stations 

Assign to each cell the value measured by each sensor. If more sensors of the same type are present in the same cell it assign the mean value:

### Meteo stations - Point layer:

In [69]:
temp_st_path = cwd + '/temp/temp_st.gpkg'
prec_st_path = cwd + '/temp/prec_st.gpkg'
air_hum_st_path = cwd + '/temp/air_hum_st.gpkg'
wind_dir_st_path = cwd + '/temp/wind_dir_st.gpkg'
wind_speed_st_path = cwd + '/temp/wind_speed_st.gpkg'
rad_glob_st_path = cwd + '/temp/rad_glob_st.gpkg'

In [70]:
temp_st = gpd.read_file(temp_st_path)
prec_st = gpd.read_file(prec_st_path)
air_hum_st = gpd.read_file(air_hum_st_path)
wind_dir_st = gpd.read_file(wind_dir_st_path)
wind_speed_st = gpd.read_file(wind_speed_st_path)
rad_glob_st = gpd.read_file(rad_glob_st_path)

In [71]:
df_dict = {'temp_st':temp_st,
          'prec_st':prec_st, 'air_hum_st': air_hum_st, 'wind_dir_st':wind_dir_st, 'wind_speed_st':wind_speed_st,
          'rad_glob_st':rad_glob_st}

In [72]:
for key in df_dict:
    grid[key] = np.nan
    poor_points = df_dict[key][['idsensore','valore','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_points, mask) 
        m = clip.valore.mean()
        grid[key].iloc[index] = m
    print(key)

temp_st
prec_st
air_hum_st
wind_dir_st
wind_speed_st
rad_glob_st


### Air quality station - Point layer:

In [73]:
pm25_st_path = cwd + '/temp/pm25_st.gpkg'
nox_st_path = cwd + '/temp/nox_st.gpkg'
no2_st_path = cwd + '/temp/no2_st.gpkg'
amm_st_path = cwd + '/temp/amm_st.gpkg'
so2_st_path = cwd + '/temp/so2_st.gpkg'
pm10_st_path = cwd + '/temp/pm10_st.gpkg'
co_st_path = cwd + '/temp/co_st.gpkg'
ozone_st_path = cwd + '/temp/o3_st.gpkg'

In [74]:
pm25_st = gpd.read_file(pm25_st_path)
nox_st = gpd.read_file(nox_st_path)
no2_st = gpd.read_file(no2_st_path)
amm_st = gpd.read_file(amm_st_path)
so2_st = gpd.read_file(so2_st_path)
pm10_st = gpd.read_file(pm10_st_path)
co_st = gpd.read_file(co_st_path)
ozone_st = gpd.read_file(ozone_st_path)

In [75]:
df_dict = {'pm25_st':pm25_st,
          'nox_st':nox_st, 'no2_st': no2_st,'ozone_st':ozone_st, 'amm_st':amm_st, 'so2_st':so2_st, 'pm10_st':pm10_st, 'co_st':co_st}

In [76]:
for key in df_dict:
    grid[key] = 0
    poor_points = df_dict[key][['idsensore','valore','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_points, mask) 
        m = clip.valore.mean()
        grid[key].iloc[index] = m
    print(key)

pm25_st
nox_st
no2_st
ozone_st
amm_st
so2_st
pm10_st
co_st


### Import Low Cost Sensor data ([ESA Air Quality Sensors](https://aqp.eo.esa.int/map/))

In [77]:
pm25_lcs_path = cwd + '/temp/pm25_lcs.gpkg'
pm10_lcs_path = cwd + '/temp/pm10_lcs.gpkg'
hum_lcs_path = cwd + '/temp/hum_lcs.gpkg'
temp_lcs_path = cwd + '/temp/temp_lcs.gpkg'
no2_lcs_path = cwd + '/temp/no2_lcs.gpkg'
co2_lcs_path = cwd + '/temp/co2_lcs.gpkg'
amm_lcs_path = cwd + '/temp/amm_lcs.gpkg'
co_lcs_path = cwd + '/temp/co_lcs.gpkg'

In [78]:
pm25_lcs = gpd.read_file(pm25_lcs_path)
pm10_lcs = gpd.read_file(pm10_lcs_path)
hum_lcs = gpd.read_file(hum_lcs_path)
temp_lcs = gpd.read_file(temp_lcs_path)
no2_lcs = gpd.read_file(no2_lcs_path)
co2_lcs = gpd.read_file(co2_lcs_path)
amm_lcs = gpd.read_file(amm_lcs_path)
co_lcs = gpd.read_file(co_lcs_path)

In [79]:
df_dict = {'pm25_lcs':pm25_lcs,
          'pm10_lcs':pm10_lcs, 'hum_lcs': hum_lcs, 'temp_lcs':temp_lcs, 'no2_lcs':no2_lcs,
          'co2_lcs':co2_lcs, 'amm_lcs':amm_lcs, 'co_lcs':co_lcs}

In [80]:
for key in df_dict:
    grid[key] = 0
    poor_points = df_dict[key][['device_id','value','geometry']]
    for index, row in grid.iterrows():
        mask = row['geometry']
        clip = gpd.clip(poor_points, mask) 
        m = clip.value.mean()
        grid[key].iloc[index] = m
    print(key)

pm25_lcs
pm10_lcs
hum_lcs
temp_lcs
no2_lcs
co2_lcs
amm_lcs
co_lcs


### Import air quality data from station interpolated data

To have a continuous information all over the grid

In [81]:
grid = grid.to_crs(32632)

In [82]:
# Set paths
int_co_path = cwd + '/temp/int_co.tif'
int_nh3_path = cwd + '/temp/int_nh3.tif'
int_no2_path = cwd + '/temp/int_no2.tif'
int_nox_path = cwd + '/temp/int_nox.tif'
int_o3_path = cwd + '/temp/int_o3.tif'
int_pm10_path = cwd + '/temp/int_pm10.tif'
int_pm25_path = cwd + '/temp/int_pm25.tif'
int_so2_path = cwd + '/temp/int_so2.tif'

int_co = rio.open(int_co_path)
int_nh3 = rio.open(int_nh3_path)
int_no2 = rio.open(int_no2_path)
int_nox = rio.open(int_nox_path)
int_o3 = rio.open(int_o3_path)
int_pm10 = rio.open(int_pm10_path)
int_pm25 = rio.open(int_pm25_path)
int_so2 = rio.open(int_so2_path)

In [83]:
int_co_array = int_co.read(1).astype('float64') 
affine = int_co.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_co_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "co_int"})

In [84]:
int_nh3_array = int_nh3.read(1).astype('float64') 
affine = int_nh3.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_nh3_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "amm_int"})

In [85]:
int_no2_array = int_no2.read(1).astype('float64') 
affine = int_no2.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_no2_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "no2_int"})

In [86]:
int_nox_array = int_nox.read(1).astype('float64') 
affine = int_nox.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_nox_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "nox_int"})

In [87]:
int_o3_array = int_o3.read(1).astype('float64') 
affine = int_o3.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_o3_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "ozone_int"})

In [88]:
int_pm10_array = int_pm10.read(1).astype('float64') 
affine = int_pm10.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_pm10_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "pm10_int"})

In [89]:
int_pm25_array = int_pm25.read(1).astype('float64') 
affine = int_pm25.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_pm25_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "pm25_int"})

In [90]:
int_so2_array = int_so2.read(1).astype('float64') 
affine = int_so2.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_so2_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "so2_int"})

### Import meteorological data from station interpolated data

In [91]:
# Set paths
int_air_hum_path = cwd + '/temp/int_air_hum.tif'
int_prec_path = cwd + '/temp/int_prec.tif'
int_rad_glob_path = cwd + '/temp/int_rad_glob.tif'
int_temp_path = cwd + '/temp/int_temp.tif'
int_wind_dir_path = cwd + '/temp/int_wind_dir.tif'
int_wind_speed_path = cwd + '/temp/int_wind_speed.tif'


int_air_hum = rio.open(int_air_hum_path)
int_prec = rio.open(int_prec_path)
int_rad_glob = rio.open(int_rad_glob_path)
int_temp = rio.open(int_temp_path)
int_wind_dir = rio.open(int_wind_dir_path)
int_wind_speed = rio.open(int_wind_speed_path)

In [92]:
int_air_hum_array = int_air_hum.read(1).astype('float64') 
affine = int_air_hum.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_air_hum_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "air_hum_int"})

In [93]:
int_prec_array = int_prec.read(1).astype('float64') 
affine = int_prec.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_prec_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "prec_int"})

In [94]:
int_rad_glob_array = int_rad_glob.read(1).astype('float64') 
affine = int_rad_glob.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_rad_glob_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "rad_glob_int"})

In [95]:
int_temp_array = int_temp.read(1).astype('float64') 
affine = int_temp.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_temp_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "temp_int"})

In [96]:
int_wind_dir_array = int_wind_dir.read(1).astype('float64') 
affine = int_wind_dir.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_wind_dir_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "wind_dir_int"})

In [97]:
int_wind_speed_array = int_wind_speed.read(1).astype('float64') 
affine = int_wind_speed.transform
grid = grid.join(pd.DataFrame(rstat.zonal_stats(grid, int_wind_speed_array, affine=affine, nodata=np.nan, stats=['mean'])), how='left')
grid = grid.rename(columns={"mean": "wind_speed_int"})

Export the file in WGS84 CRS:

In [98]:
grid.to_crs(4326).to_file(cwd+"/results/test_grid.gpkg", driver="GPKG")

In [99]:
grid

,left,bottom,right,top,lng_cen,lat_cen,geometry,dusaf,dsf2,dsf3,...,ozone_int,pm10_int,pm25_int,so2_int,air_hum_int,prec_int,rad_glob_int,temp_int,wind_dir_int,wind_speed_int
0,8.2,44.4,8.3,44.5,8.25,44.45,"MULTIPOLYGON (((436293.144 4916612.712, 436401...",NaN,NaN,NaN,...,66.890655,31.298317,28.146694,6.635157,101.096294,0.009789,155.390797,5.115451,228.650395,5.666876
1,8.2,44.5,8.3,44.6,8.25,44.55,"MULTIPOLYGON (((436401.760 4927720.393, 436510...",NaN,NaN,NaN,...,64.904646,30.456077,27.245959,6.502302,98.482582,0.009746,151.952397,5.310015,223.521301,5.377196
2,8.2,44.6,8.3,44.7,8.25,44.65,"MULTIPOLYGON (((436510.572 4938828.265, 436619...",NaN,NaN,NaN,...,62.753939,29.565967,26.231054,6.351002,95.715548,0.009681,148.230583,5.571837,218.251631,5.034347
3,8.2,44.7,8.3,44.8,8.25,44.75,"MULTIPOLYGON (((436619.577 4949936.329, 436728...",NaN,NaN,NaN,...,60.835354,28.819851,25.273416,6.198818,93.302422,0.009588,144.820944,5.859611,213.938941,4.693400
4,8.2,44.8,8.3,44.9,8.25,44.85,"MULTIPOLYGON (((436728.778 4961044.585, 436838...",NaN,NaN,NaN,...,58.939218,28.174536,24.269558,6.012602,90.921483,0.009439,141.190677,6.190218,210.150581,4.317827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,11.7,46.4,11.8,46.5,11.75,46.45,"MULTIPOLYGON (((707551.942 5142034.521, 707172...",NaN,NaN,NaN,...,82.384258,26.346387,31.968855,3.771769,102.630929,0.006818,147.063889,1.883040,239.998497,5.145190
896,11.7,46.5,11.8,46.6,11.75,46.55,"MULTIPOLYGON (((707172.174 5153145.473, 706791...",NaN,NaN,NaN,...,85.451675,26.307562,32.876406,3.921079,105.554519,0.006606,151.868273,1.220466,246.428140,5.426224
897,11.7,46.6,11.8,46.7,11.75,46.65,"MULTIPOLYGON (((706791.772 5164256.577, 706410...",NaN,NaN,NaN,...,88.222382,26.412053,33.700753,4.065596,108.370142,0.006426,156.497997,0.698046,252.763082,5.692212
898,11.7,46.7,11.8,46.8,11.75,46.75,"MULTIPOLYGON (((706410.737 5175367.832, 706029...",NaN,NaN,NaN,...,90.759602,26.563815,34.458274,4.201840,111.007278,0.006259,160.922479,0.235164,258.904127,5.947217
